In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_parquet('processed_data/net_flow_model_table.parquet')
df.head()

,station_id,time_bucket,hour,dow,net_flow_lag_1,net_flow_lag_2,net_flow_lag_3,net_flow_lag_6,roll_mean_3,roll_std_3,target
0,000959,2022-09-12 11:00:00,11,0,-1.0,-1.0,1.0,0.0,-0.333333,1.154701,balanced
1,000959,2022-09-12 11:30:00,11,0,1.0,-1.0,-1.0,5.0,-0.333333,1.154701,balanced
2,000959,2022-09-12 12:00:00,12,0,1.0,1.0,-1.0,-1.0,0.333333,1.154701,balanced
3,000959,2022-09-12 12:30:00,12,0,-2.0,1.0,1.0,1.0,0.000000,1.732051,balanced
4,000959,2022-09-12 13:00:00,13,0,2.0,-2.0,1.0,-1.0,0.333333,2.081666,balanced


In [3]:
df['target'].value_counts()

target
balanced    17126824
surplus       321574
shortage      296246
Name: count, dtype: int64

- Can't use train_test_split from sklearn on this data since it's time series data therefore opted for TimeSeriesSplit from sklearn. 
- Make sure to sort data by TIME. 

In [4]:
#split data into training & testing set
from sklearn.model_selection import TimeSeriesSplit 

df = df.sort_values('time_bucket').reset_index(drop = True) 

X = df.drop(columns = ['station_id', 'time_bucket', 'target'])
y = df['target']

tscv = TimeSeriesSplit(n_splits = 3) #3 splits only because 5 seems to crash computer

In [5]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def build_model():
    return LogisticRegression(
        multi_class = 'multinomial',
        max_iter = 500,
        class_weight = 'balanced', 
        n_jobs = -1,
        solver = 'lbfgs'
    )

reports = []

for fold, (train_idx, test_idx) in enumerate(tscv.split(X), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = build_model()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)

    print(f'{fold}')
    print(classification_report(y_test, y_pred, digits = 3))

    reports.append(
        classification_report(
            y_test, y_pred, output_dict = True
        )
    )


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


1
              precision    recall  f1-score   support

    balanced      0.981     0.505     0.667   4281437
    shortage      0.035     0.537     0.065     74334
     surplus      0.043     0.575     0.080     80390

    accuracy                          0.507   4436161
   macro avg      0.353     0.539     0.270   4436161
weighted avg      0.948     0.507     0.646   4436161



/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


2
              precision    recall  f1-score   support

    balanced      0.979     0.504     0.665   4271529
    shortage      0.035     0.522     0.066     79511
     surplus      0.045     0.567     0.084     85121

    accuracy                          0.505   4436161
   macro avg      0.353     0.531     0.272   4436161
weighted avg      0.945     0.505     0.644   4436161



/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


3
              precision    recall  f1-score   support

    balanced      0.982     0.523     0.683   4281449
    shortage      0.036     0.543     0.068     74193
     surplus      0.047     0.611     0.088     80519

    accuracy                          0.525   4436161
   macro avg      0.355     0.559     0.279   4436161
weighted avg      0.949     0.525     0.662   4436161



In [6]:
#aggregate cross-val performance
cv_summary = (
    pd.DataFrame([
        {
            "fold": i + 1,
            "shortage_recall": r["shortage"]["recall"],
            "surplus_recall":  r["surplus"]["recall"],
            "macro_f1":        r["macro avg"]["f1-score"],
        }
        for i, r in enumerate(reports)
    ])
)

print(cv_summary)
cv_summary.mean()


   fold  shortage_recall  surplus_recall  macro_f1
0     1         0.536753        0.575146  0.270455
1     2         0.521777        0.567475  0.271858
2     3         0.543232        0.611346  0.279407


fold               2.000000
shortage_recall    0.533921
surplus_recall     0.584656
macro_f1           0.273906
dtype: float64

- model correctly identifies over half of all real shortages and surpluses 30 min in advance, using only real net flow history and time features.
- macro f1 relatively low but that's ok because it averages performance across three classes: shortage (rare), surplus (rare) and balanced (dominant). 96.5% of cases are balanced while minority classes are ~1-2% so macro f1 is going be low. 

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf_reports = []

for fold, (train_idx, test_idx) in enumerate(tscv.split(X), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    rf = RandomForestClassifier(
        n_estimators=80,
        max_depth=8,
        min_samples_leaf=100,
        class_weight="balanced",
        n_jobs=2,
        random_state=42
    )

    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    print(f'{fold}')
    print(classification_report(y_test, y_pred, digits=3))

    rf_reports.append(
        classification_report(y_test, y_pred, output_dict=True)
    )


1
              precision    recall  f1-score   support

    balanced      0.985     0.684     0.807   4281437
    shortage      0.049     0.577     0.090     74334
     surplus      0.074     0.541     0.131     80390

    accuracy                          0.679   4436161
   macro avg      0.369     0.601     0.343   4436161
weighted avg      0.953     0.679     0.783   4436161

2
              precision    recall  f1-score   support

    balanced      0.983     0.680     0.804   4271529
    shortage      0.050     0.558     0.092     79511
     surplus      0.076     0.533     0.133     85121

    accuracy                          0.675   4436161
   macro avg      0.370     0.591     0.343   4436161
weighted avg      0.949     0.675     0.779   4436161

3
              precision    recall  f1-score   support

    balanced      0.986     0.691     0.813   4281449
    shortage      0.052     0.595     0.095     74193
     surplus      0.080     0.582     0.141     80519

    accuracy  

In [9]:
rf_summary = pd.DataFrame([
    {
        "fold": i + 1,
        "shortage_recall": r["shortage"]["recall"],
        "surplus_recall":  r["surplus"]["recall"],
        "macro_f1":        r["macro avg"]["f1-score"],
    }
    for i, r in enumerate(rf_reports)
])

print(rf_summary)
rf_summary.mean()


   fold  shortage_recall  surplus_recall  macro_f1
0     1         0.576667        0.541398  0.342579
1     2         0.558111        0.533405  0.343245
2     3         0.594760        0.581776  0.349674


fold               2.000000
shortage_recall    0.576513
surplus_recall     0.552193
macro_f1           0.345166
dtype: float64

Compared to Logisitic Regression, the Random Forest saw:
- 4-5pp improvement in shortage recall 
- Macro F1 up by approx 0.07 (relatively big jump in an imbalanced 3 class problem)
- Shortage recall is most important metric because shortages are more operationally costly so the improved shortage recall (53% to 57%) is meaningful. 

In [11]:
label_map = {
    "shortage": 0,
    "balanced": 1,
    "surplus":  2
}

y_enc = df["target"].map(label_map)
y_enc.value_counts()

target
1    17126824
2      321574
0      296246
Name: count, dtype: int64

In [13]:
import xgboost as xgb 

xgb_reports = []

for fold, (train_idx, test_idx) in enumerate(tscv.split(X), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y_enc.iloc[train_idx], y_enc.iloc[test_idx]

    model = xgb.XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        n_estimators=150,        
        max_depth=5,             # shallow trees
        learning_rate=0.1,
        subsample=0.7,
        colsample_bytree=0.7,
        tree_method="hist",      # memory-efficient
        eval_metric="mlogloss",
        n_jobs=2,                # NOT all cores
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f'{fold}')
    print(
        classification_report(
            y_test,
            y_pred,
            target_names=["shortage", "balanced", "surplus"],
            digits=3
        )
    )

    xgb_reports.append(
        classification_report(
            y_test,
            y_pred,
            output_dict=True
        )
    )



1
              precision    recall  f1-score   support

    shortage      0.781     0.016     0.032     74334
    balanced      0.966     1.000     0.983   4281437
     surplus      0.691     0.038     0.072     80390

    accuracy                          0.966   4436161
   macro avg      0.813     0.351     0.362   4436161
weighted avg      0.958     0.966     0.950   4436161

2
              precision    recall  f1-score   support

    shortage      0.737     0.016     0.032     79511
    balanced      0.964     1.000     0.981   4271529
     surplus      0.709     0.046     0.087     85121

    accuracy                          0.964   4436161
   macro avg      0.803     0.354     0.367   4436161
weighted avg      0.955     0.964     0.947   4436161

3
              precision    recall  f1-score   support

    shortage      0.718     0.019     0.036     74193
    balanced      0.966     0.999     0.983   4281449
     surplus      0.684     0.053     0.098     80519

    accuracy  

In [15]:
xgb_summary = pd.DataFrame([
    {
        "fold": i + 1,
        "shortage_recall": r["0"]["recall"],   # 0 -> shortage
        "surplus_recall":  r["2"]["recall"],   # 2 -> surplus
        "macro_f1":        r["macro avg"]["f1-score"],
    }
    for i, r in enumerate(xgb_reports)
])

print(xgb_summary)
xgb_summary.mean()


   fold  shortage_recall  surplus_recall  macro_f1
0     1         0.016372        0.037990  0.362210
1     2         0.016463        0.046087  0.366733
2     3         0.018600        0.052832  0.372316


fold               2.000000
shortage_recall    0.017145
surplus_recall     0.045636
macro_f1           0.367086
dtype: float64

- XGBoost Model failed as it fails to predict shortages or surpluses. Instead it overwhelmingly predicts the balanced class.  
- Reject this model. 

In [16]:
best_model = RandomForestClassifier(
    n_estimators=120, #slightly increase n_estimators (from 80)
    max_depth=8,
    min_samples_leaf=100,
    class_weight="balanced",
    n_jobs=2,
    random_state=42
)

best_model.fit(X, y)

#make predictions
rf_preds = df[["station_id", "time_bucket"]].copy()
rf_preds["predicted_status"] = best_model.predict(X)

rf_preds.head()
rf_preds["predicted_status"].value_counts(normalize=True)


predicted_status
balanced    0.674875
shortage    0.197840
surplus     0.127285
Name: proportion, dtype: float64

In [18]:
from pathlib import Path 

#save predictions
out_dir = Path('processed_data')
out_dir.mkdir(exist_ok = True)

rf_preds.to_parquet(
    out_dir / 'rf_predictions.parquet',
    index = False
)

test = pd.read_parquet("processed_data/rf_predictions.parquet")
test.shape
test["station_id"].nunique()
test["predicted_status"].value_counts(normalize = True)


predicted_status
balanced    0.674875
shortage    0.197840
surplus     0.127285
Name: proportion, dtype: float64